# Analisis General - Simulación Base

## Pre-procesamiento

In [4]:
import pandas as pd

# 1. Leer los archivos
arma_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_ARMA_FINAL.xlsx")
arima_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_ARIMA_FINAL.xlsx")
setar_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_SETAR_FINAL.xlsx")

# 2. Asignar la columna ESCENARIO a cada dataframe
arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# 3. Juntarlos uno bajo el otro (Concatenar)
df_total = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Seleccionar solo las columnas requeridas
columnas_deseadas = [
    "Paso", "Config", "Dist", "Var", "Block Bootstrapping", 
    "Sieve Bootstrap", "LSPM", "LSPMW", "AREPD", "MCPS", 
    "AV-MCPS", "DeepAR", "EnCQR-LSTM", "ESCENARIO"
]
df_total = df_total[columnas_deseadas]

# Definimos cuáles son las columnas que representan a los modelos predictivos
modelos = [
    "Block Bootstrapping", "Sieve Bootstrap", "LSPM", "LSPMW", 
    "AREPD", "MCPS", "AV-MCPS", "DeepAR", "EnCQR-LSTM"
]

# 4. Guardar el dataframe consolidado
df_total.to_excel("./datos/Simulacion/Base/dataframe_consolidado.xlsx", index=False)

# 5. Generar y mostrar las tablas (Media y Mediana)
metricas = {'MEDIA': 'mean', 'MEDIANA': 'median'}

for nombre_metrica, funcion in metricas.items():
    # Calculamos el valor general según la métrica (mean o median)
    if funcion == 'mean':
        resumen_general = df_total[modelos].mean()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].mean().T
    else:
        resumen_general = df_total[modelos].median()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].median().T

    # Construimos la tabla final para esta métrica
    tabla_resumen = pd.DataFrame(index=modelos)
    tabla_resumen['General'] = resumen_general
    tabla_resumen['ARMA'] = resumen_escenarios['Lineal Estacionario']
    tabla_resumen['ARIMA'] = resumen_escenarios['Lineal No estacionario']
    tabla_resumen['SETAR'] = resumen_escenarios['No lineal Estacionario']

    # Determinar el Mejor_Escenario (valor mínimo entre los tres escenarios)
    escenarios_cols = ['ARMA', 'ARIMA', 'SETAR']
    tabla_resumen['Mejor_Escenario'] = tabla_resumen[escenarios_cols].idxmin(axis=1)

    # Imprimir resultado
    print(f"\n--- Tabla Comparativa de Modelos (Basada en {nombre_metrica}) ---")
    print(tabla_resumen.reset_index().rename(columns={'index': 'Modelo'}).to_string(index=False))


--- Tabla Comparativa de Modelos (Basada en MEDIA) ---
             Modelo  General     ARMA     ARIMA    SETAR Mejor_Escenario
Block Bootstrapping 4.275365 0.946781 11.251601 0.627714           SETAR
    Sieve Bootstrap 0.570491 0.547725  0.547481 0.616268           ARIMA
               LSPM 0.845497 0.811287  1.064804 0.660398           SETAR
              LSPMW 1.572319 0.960743  3.079645 0.676570           SETAR
              AREPD 3.880255 0.936671 10.031183 0.672910           SETAR
               MCPS 1.553139 0.752295  3.218168 0.688955           SETAR
            AV-MCPS 1.578091 0.733188  3.324007 0.677078           SETAR
             DeepAR 1.836024 0.568693  4.329124 0.610255            ARMA
         EnCQR-LSTM 2.675191 1.071557  6.112344 0.841671           SETAR

--- Tabla Comparativa de Modelos (Basada en MEDIANA) ---
             Modelo  General     ARMA    ARIMA    SETAR Mejor_Escenario
Block Bootstrapping 0.989415 0.654169 5.283530 0.540627           SETAR
    Sieve Bo

## Analisis general

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest
import warnings
from pathlib import Path
import itertools

warnings.filterwarnings('ignore')

# Configuración general
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
sns.set_palette("husl")

# Crear carpeta de resultados
output_dir = Path("./Resultados_analisis/Simulacion_base")
output_dir.mkdir(parents=True, exist_ok=True)

# Crear carpeta de interacciones
interactions_dir = output_dir / "Interacciones"
interactions_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_excel("./datos/Simulacion/Base/dataframe_consolidado.xlsx")

# 1) CAMBIO DE NOMBRES DE ESCENARIOS
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario": "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

# Identificar columnas de modelos
var_cols = ['Paso', 'Config', 'Dist', 'Var', 'ESCENARIO']
original_model_cols = [col for col in df.columns if col not in var_cols]

# 2) ORGANIZAR MODELOS POR RENDIMIENTO EN "Lineal No Estacionario (ARIMA)" (Menor a mayor)
target_scenario = "Lineal No Estacionario (ARIMA)"
model_order_scores = df[df['ESCENARIO'] == target_scenario][original_model_cols].mean().sort_values()
model_cols = list(model_order_scores.index)

print("Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):")
for i, m in enumerate(model_cols, 1):
    print(f"{i}. {m}: {model_order_scores[m]:.4f}")

# Mapeo de escenarios
escenarios_map = {
    'Lineal Estacionario (ARMA)': 'Lineal Estacionario (ARMA)',
    'Lineal No Estacionario (ARIMA)': 'Lineal No Estacionario (ARIMA)',
    'No lineal Estacionario (SETAR)': 'No lineal Estacionario (SETAR)'
}

# Definir colores para cada modelo
palette = sns.color_palette("husl", len(model_cols))
model_colors = {model: palette[i] for i, model in enumerate(model_cols)}

# ====================================================================================
# SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 1: RENDIMIENTO POR ESCENARIOS")
print("="*80)

def plot_performance_by_scenario():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    scenarios = ["Lineal Estacionario (ARMA)", "Lineal No Estacionario (ARIMA)", "No lineal Estacionario (SETAR)"]
    x = np.arange(len(model_cols))
    width = 0.25 
    
    scenario_colors = {
        'Lineal Estacionario (ARMA)': '#5D3FD3',    
        'Lineal No Estacionario (ARIMA)': '#808080', 
        'No lineal Estacionario (SETAR)': '#00A36C'  
    }
    
    for idx, scenario in enumerate(scenarios):
        means = [df[df['ESCENARIO'] == scenario][model].mean() for model in model_cols]
        position = x + (idx - 1) * width
        
        bars = ax.bar(position, means, width, label=scenario, 
                     color=scenario_colors[scenario], alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=7, rotation=0)
    
    ax.set_xlabel('Modelo (Ordenados por desempeño en ARIMA)', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title('Rendimiento de Modelos por Escenario (ECRPS)', fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.legend(loc='upper left', ncol=1, fontsize=10, framealpha=0.9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.1_rendimiento_por_escenario.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.1 guardada")

plot_performance_by_scenario()

def plot_relative_performance():
    base_scenario = 'Lineal Estacionario (ARMA)'
    scenarios_compare = ['Lineal No Estacionario (ARIMA)', 'No lineal Estacionario (SETAR)']
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    y = np.arange(len(model_cols))
    height = 0.35  
    
    for idx, scenario in enumerate(scenarios_compare):
        changes = []
        for model in model_cols:
            base_value = df[df['ESCENARIO'] == base_scenario][model].mean()
            scenario_value = df[df['ESCENARIO'] == scenario][model].mean()
            pct_change = ((scenario_value - base_value) / base_value) * 100
            changes.append(pct_change)
        
        position = y + idx * height
        bars = ax.barh(position, changes, height, label=scenario, alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar, val in zip(bars, changes):
            width = bar.get_width()
            ax.text(width + (1 if width > 0 else -1), bar.get_y() + bar.get_height()/2.,
                   f'{val:+.1f}%', ha='left' if val > 0 else 'right', 
                   va='center', fontsize=7)
    
    ax.set_yticks(y + height / 2)
    ax.set_yticklabels(model_cols, fontsize=10)
    ax.set_xlabel('Cambio Relativo (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title(f'Cambio Relativo en ECRPS vs. {base_scenario}', fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1.5)
    ax.legend(loc='best', fontsize=10, framealpha=0.9)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.2_cambio_relativo_escenario_base.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.2 guardada")

plot_relative_performance()

# ====================================================================================
# SECCIÓN 2: ANÁLISIS POR CONFIG
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 2: ANÁLISIS POR CONFIG")
print("="*80)

def plot_zscore_heatmap_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Configuración (General)'
    
    pivot_data = data_filtered.groupby('Config')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'2.1{suffix}_zscore_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.1{suffix} guardada")

plot_zscore_heatmap_config()
plot_zscore_heatmap_config('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_config('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_config('No lineal Estacionario (SETAR)', '.c')

def plot_variability_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Configuración (General)'
    
    pivot_std = data_filtered.groupby('Config')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    fmt = '.2f' if suffix == '' else '.4f'
    sns.heatmap(pivot_std.T, annot=True, fmt=fmt, cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'2.2{suffix}_variabilidad_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.2{suffix} guardada")

plot_variability_config()
plot_variability_config('Lineal Estacionario (ARMA)', '.a')
plot_variability_config('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_config('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 3: ANÁLISIS POR DIST
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 3: ANÁLISIS POR DIST")
print("="*80)

def plot_zscore_heatmap_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Distribución (General)'
    
    pivot_data = data_filtered.groupby('Dist')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'3.1{suffix}_zscore_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.1{suffix} guardada")

plot_zscore_heatmap_dist()
plot_zscore_heatmap_dist('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_dist('No lineal Estacionario (SETAR)', '.c')

def plot_variability_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Distribución (General)'
    
    pivot_std = data_filtered.groupby('Dist')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'3.2{suffix}_variabilidad_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.2{suffix} guardada")

plot_variability_dist()
plot_variability_dist('Lineal Estacionario (ARMA)', '.a')
plot_variability_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_dist('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 4: ANÁLISIS POR VAR
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 4: ANÁLISIS POR VAR")
print("="*80)

def plot_evolution_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Varianza (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    var_values = sorted(data_filtered['Var'].unique())
    
    for model in model_cols:
        means = []
        for var in var_values:
            mean_val = data_filtered[data_filtered['Var'] == var][model].mean()
            means.append(mean_val)
        
        ax.plot(var_values, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'4.1{suffix}_evolucion_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.1{suffix} guardada")

plot_evolution_var()
plot_evolution_var('Lineal Estacionario (ARMA)', '.a')
plot_evolution_var('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_var('No lineal Estacionario (SETAR)', '.c')

def plot_variability_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Varianza (General)'
    
    pivot_std = data_filtered.groupby('Var')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'4.2{suffix}_variabilidad_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.2{suffix} guardada")

plot_variability_var()
plot_variability_var('Lineal Estacionario (ARMA)', '.a')
plot_variability_var('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_var('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)")
print("="*80)

def plot_evolution_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Horizonte de Pronóstico ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Horizonte de Pronóstico (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    pasos = sorted(data_filtered['Paso'].unique())
    
    for model in model_cols:
        means = []
        for paso in pasos:
            mean_val = data_filtered[data_filtered['Paso'] == paso][model].mean()
            means.append(mean_val)
        
        ax.plot(pasos, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'5.1{suffix}_evolucion_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.1{suffix} guardada")

plot_evolution_paso()
plot_evolution_paso('Lineal Estacionario (ARMA)', '.a')
plot_evolution_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_paso('No lineal Estacionario (SETAR)', '.c')

def plot_variability_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Horizonte ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Horizonte (General)'
    
    pivot_std = data_filtered.groupby('Paso')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'5.2{suffix}_variabilidad_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.2{suffix} guardada")

plot_variability_paso()
plot_variability_paso('Lineal Estacionario (ARMA)', '.a')
plot_variability_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_paso('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES")
print("="*80)

# Función auxiliar para configurar el grid de modelos
def get_model_grid_axes(n_models):
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
    return fig, axes.flatten(), n_rows, n_cols

# INTERACCIÓN 3 (Lista 8): DIST × PASO - Subplots por Modelo
def plot_interaction_dist_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, n_rows, n_cols = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_dist = sns.color_palette("viridis", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='o', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.7{suffix}_interaccion_dist_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 8 (5.7) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 4 (Lista 9): DIST × VAR - Subplots por Modelo
def plot_interaction_dist_var(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Var por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    vars_val = sorted(data_filtered['Var'].unique())
    colors_dist = sns.color_palette("magma", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Var'] == v)][model].mean() for v in vars_val]
            ax.plot(vars_val, means, marker='s', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Varianza')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.8{suffix}_interaccion_dist_var_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 9 (5.8) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 5 (Lista 10): CONFIG × PASO - Subplots por Modelo
def plot_interaction_config_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Config × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    configs = sorted(data_filtered['Config'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_conf = sns.color_palette("tab10", len(configs))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for c_idx, config in enumerate(configs):
            means = [data_filtered[(data_filtered['Config'] == config) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='^', label=config, color=colors_conf[c_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Config", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.9{suffix}_interaccion_config_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 10 (5.9) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 6 (Lista 11): VAR × HORIZONTE - Subplots por Modelo
def plot_interaction_var_horizonte(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Var × Horizonte por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    vars_val = sorted(data_filtered['Var'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_var = sns.color_palette("rocket", len(vars_val))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for v_idx, var in enumerate(vars_val):
            means = [data_filtered[(data_filtered['Var'] == var) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='d', label=f'Var {var}', color=colors_var[v_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Varianza", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.10{suffix}_interaccion_var_horizonte_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 11 (5.10) {suffix} guardada: Subplots por Modelo")


# ====================================================================================
# EJECUCIÓN DE LAS NUEVAS FUNCIONES
# ====================================================================================

for sc_name, sc_suf in [ (None, ''), ('Lineal Estacionario (ARMA)', '.a'), 
                        ('Lineal No Estacionario (ARIMA)', '.b'), 
                        ('No lineal Estacionario (SETAR)', '.c') ]:
    plot_interaction_dist_paso(sc_name, sc_suf)
    plot_interaction_dist_var(sc_name, sc_suf)
    plot_interaction_config_paso(sc_name, sc_suf)
    plot_interaction_var_horizonte(sc_name, sc_suf)

# ====================================================================================
# SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO")
print("="*80)

def plot_robustness():
    fig, ax = plt.subplots(figsize=(12, 8))
    
    cv_data = []
    for model in model_cols:
        cv = df[model].std() / df[model].mean()
        cv_data.append((model, cv))
    
    cv_df = pd.DataFrame(cv_data, columns=['Modelo', 'CV'])
    
    # CAMBIO 1: Ordenar de menor a mayor CV para coherencia
    cv_df = cv_df.sort_values('CV')
    
    colors_cv = ['#2ecc71' if cv < cv_df['CV'].median() else '#e74c3c' 
                 for cv in cv_df['CV']]
    
    bars = ax.barh(cv_df['Modelo'], cv_df['CV'], color=colors_cv, alpha=0.8, edgecolor='black')
    
    for bar, cv in zip(bars, cv_df['CV']):
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2.,
               f'{cv:.4f}', ha='left', va='center', fontsize=9)
    
    ax.set_xlabel('Coeficiente de Variación', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title('Robustez: Coeficiente de Variación\n(Ordenado de menor a mayor - Menor valor indica mayor estabilidad)', 
                  fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=cv_df['CV'].median(), color='black', linestyle='--', linewidth=1.5, alpha=0.5, label='Mediana')
    ax.legend(loc='best', fontsize=10)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '6.1_robustez_coeficiente_variacion.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 6.1 guardada (ordenada de menor a mayor CV)")

plot_robustness()

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano modificado con corrección Harvey-Leybourne-Newbold (1997)
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hlm_dm_stat : float
        Estadístico DM corregido (HLN-DM)
    p_value : float
        P-valor usando distribución t-Student con T-1 grados de libertad
    dm_stat : float
        Estadístico DM original (sin corrección)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    # Calcular autocovarianzas
    def gamma_d(k):
        if k == 0:
            return np.var(d, ddof=1)
        else:
            return np.mean((d[k:] - d_bar) * (d[:-k] - d_bar))
    
    # Estimar la varianza de largo plazo usando Newey-West
    # Para h-step-ahead forecasts, incluimos hasta h-1 lags
    gamma_0 = gamma_d(0)
    gamma_sum = gamma_0
    
    if h > 1:
        for k in range(1, h):
            gamma_k = gamma_d(k)
            gamma_sum += 2 * gamma_k
    
    var_d = gamma_sum / T
    
    if var_d <= 0:
        return 0, 1.0, 0
    
    # Estadístico DM original
    dm_stat = d_bar / np.sqrt(var_d)
    
    # Corrección Harvey-Leybourne-Newbold (1997)
    correction_factor = np.sqrt((T + 1 - 2*h + h*(h-1)) / T)
    hln_dm_stat = correction_factor * dm_stat
    
    # P-valor usando t-Student con T-1 grados de libertad
    df = T - 1
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

def plot_dm_test_heatmap(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Test Diebold-Mariano Modificado (HLN-DM)\ncon Corrección de Bonferroni ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Test Diebold-Mariano Modificado (HLN-DM)\ncon Corrección de Bonferroni (General)'
    
    # Determinar el horizonte de pronóstico promedio
    h_forecast = int(data_filtered['Paso'].mean())
    
    n_models = len(model_cols)
    n_comparisons = n_models * (n_models - 1) / 2
    alpha = 0.05
    bonferroni_alpha = alpha / n_comparisons
    
    results_matrix = np.zeros((n_models, n_models))
    p_values = np.zeros((n_models, n_models))
    dm_stats = np.zeros((n_models, n_models))
    
    for i, model1 in enumerate(model_cols):
        for j, model2 in enumerate(model_cols):
            if i == j:
                results_matrix[i, j] = 0  
                p_values[i, j] = 1.0
                dm_stats[i, j] = 0
            elif i < j:
                errors1 = data_filtered[model1].values
                errors2 = data_filtered[model2].values
                hln_dm_stat, p_val, dm_original = modified_diebold_mariano_test(
                    errors1, errors2, h=h_forecast
                )
                
                p_values[i, j] = p_val
                p_values[j, i] = p_val
                dm_stats[i, j] = hln_dm_stat
                dm_stats[j, i] = -hln_dm_stat
                
                if p_val < bonferroni_alpha:
                    mean1 = np.mean(errors1)
                    mean2 = np.mean(errors2)
                    if mean1 < mean2:  
                        results_matrix[i, j] = 1  
                        results_matrix[j, i] = -1  
                    else:
                        results_matrix[i, j] = -1
                        results_matrix[j, i] = 1
                else:
                    results_matrix[i, j] = 0
                    results_matrix[j, i] = 0
    
    fig, ax = plt.subplots(figsize=(14, 11))
    cmap = plt.cm.colors.ListedColormap(['#e74c3c', '#fff9c4', '#2ecc71'])
    bounds = [-1.5, -0.5, 0.5, 1.5]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
    
    im = ax.imshow(results_matrix, cmap=cmap, norm=norm, aspect='auto')
    ax.set_xticks(np.arange(n_models))
    ax.set_yticks(np.arange(n_models))
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.set_yticklabels(model_cols, fontsize=9)
    
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                text = '-'
                color = 'black'
            else:
                val = results_matrix[i, j]
                p_val = p_values[i, j]
                dm_val = dm_stats[i, j]
                if val == 1:
                    text = f'✓\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                elif val == -1:
                    text = f'✗\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                else:
                    text = f'≈\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'black'
            ax.text(j, i, text, ha='center', va='center', 
                   color=color, fontsize=6, fontweight='bold')
    
    # Obtener T para el título
    T = len(data_filtered)
    df_test = T - 1
    
    ax.set_title(title + f'\n(h={h_forecast}, T={T}, df={df_test}, α ajustado={bonferroni_alpha:.5f})', 
                 fontsize=11, fontweight='bold', pad=20)
    ax.set_xlabel('Modelo (Columna)', fontsize=11)
    ax.set_ylabel('Modelo (Fila)', fontsize=11)
    
    legend_elements = [
        plt.Rectangle((0,0),1,1, facecolor='#2ecc71', label='Fila supera columna (p < α)'),
        plt.Rectangle((0,0),1,1, facecolor='#e74c3c', label='Columna supera fila (p < α)'),
        plt.Rectangle((0,0),1,1, facecolor='#fff9c4', label='Sin diferencia significativa (p ≥ α)')
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=9)
    
    # Agregar nota metodológica
    note_text = ('Nota: Se utiliza el test HLN-DM con corrección para muestras finitas.\n'
                 'Distribución: t-Student. Ajuste: Bonferroni para comparaciones múltiples.')
    plt.figtext(0.5, -0.02, note_text, ha='center', fontsize=8, style='italic', 
                wrap=True, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    filename = f'6.2{suffix}_dm_test_hln_bonferroni.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    
    # Guardar resultados en Excel
    results_df = pd.DataFrame(results_matrix, index=model_cols, columns=model_cols)
    pvalues_df = pd.DataFrame(p_values, index=model_cols, columns=model_cols)
    dmstats_df = pd.DataFrame(dm_stats, index=model_cols, columns=model_cols)
    
    summary_data = []
    for model in model_cols:
        idx = model_cols.index(model)
        victorias = int(np.sum(results_matrix[idx, :] == 1))
        derrotas = int(np.sum(results_matrix[idx, :] == -1))
        empates = int(np.sum(results_matrix[idx, :] == 0)) - 1
        mean_ecrps = data_filtered[model].mean()
        std_ecrps = data_filtered[model].std()
        cv_ecrps = std_ecrps / mean_ecrps
        summary_data.append({
            'Modelo': model,
            'Victorias': victorias,
            'Derrotas': derrotas,
            'Empates': empates,
            'Tasa_Victoria': f"{(victorias / (n_models - 1)) * 100:.1f}%",
            'ECRPS_Promedio': f"{mean_ecrps:.4f}",
            'ECRPS_Desv_Std': f"{std_ecrps:.4f}",
            'Coef_Variacion': f"{cv_ecrps:.4f}"
        })
    summary_df = pd.DataFrame(summary_data)
    
    # Información metodológica
    method_info = pd.DataFrame({
        'Parámetro': ['Horizonte de pronóstico (h)', 'Tamaño muestra (T)', 
                      'Grados libertad (df)', 'Alpha nominal', 
                      'Alpha ajustado (Bonferroni)', 'Número comparaciones',
                      'Distribución', 'Corrección aplicada'],
        'Valor': [h_forecast, T, df_test, alpha, bonferroni_alpha, 
                  int(n_comparisons), 't-Student', 'Harvey-Leybourne-Newbold (1997)']
    })
    
    with pd.ExcelWriter(output_dir / f'6.2{suffix}_dm_test_hln_resultados.xlsx') as writer:
        method_info.to_excel(writer, sheet_name='Metodologia', index=False)
        results_df.to_excel(writer, sheet_name='Matriz_Resultados')
        pvalues_df.to_excel(writer, sheet_name='P_valores')
        dmstats_df.to_excel(writer, sheet_name='Estadisticos_HLN_DM')
        summary_df.to_excel(writer, sheet_name='Resumen_Modelos', index=False)
    
    print(f"✓ Gráfica 6.2{suffix} guardada (Test HLN-DM con h={h_forecast}, T={T}, df={df_test})")

plot_dm_test_heatmap()
plot_dm_test_heatmap('Lineal Estacionario (ARMA)', '.a')
plot_dm_test_heatmap('Lineal No Estacionario (ARIMA)', '.b')
plot_dm_test_heatmap('No lineal Estacionario (SETAR)', '.c')

print("\n" + "="*80)
print("PROCESO COMPLETADO")
print("="*80)
print("\nMejoras implementadas:")
print("1. ✓ Orden consistente en gráficas 1.1 y 1.2")
print("2. ✓ Formato de 2 decimales en heatmap 2.2 general")
print("3. ✓ Solo Coeficiente de Variación en gráfica 6.1 (ordenado menor a mayor)")
print("4. ✓ Test Diebold-Mariano con corrección HLN y distribución t-Student")
print("5. ✓ Horizonte de pronóstico (h) incorporado en el análisis")
print("\n" + "="*80)
print("ANÁLISIS DE INTERACCIONES (Carpeta: Interacciones/)")
print("="*80)
print("6. ✓ Config × Var: Heatmaps por modelo (matriz 3×3)")
print("7. ✓ Config × Dist: Heatmaps por modelo (matriz 3×3)")
print("8. ✓ Dist × Paso: Gráficas de líneas por distribución")
print("9. ✓ Dist × Var: Gráficas de líneas por distribución")
print("10. ✓ Config × Paso: Gráficas de líneas por configuración")
print("11. ✓ Var × Horizonte: Gráficas de líneas por varianza")
print("="*80)
print(f"\nTotal de gráficas generadas en Interacciones/: {6 * 4} archivos")
print("(6 tipos de interacción × 4 escenarios: General + 3 específicos)")
print("="*80)

Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):
1. Sieve Bootstrap: 0.5475
2. LSPM: 1.0648
3. LSPMW: 3.0796
4. MCPS: 3.2182
5. AV-MCPS: 3.3240
6. DeepAR: 4.3291
7. EnCQR-LSTM: 6.1123
8. AREPD: 10.0312
9. Block Bootstrapping: 11.2516

SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
✓ Gráfica 1.1 guardada
✓ Gráfica 1.2 guardada

SECCIÓN 2: ANÁLISIS POR CONFIG
✓ Gráfica 2.1 guardada
✓ Gráfica 2.1.a guardada
✓ Gráfica 2.1.b guardada
✓ Gráfica 2.1.c guardada
✓ Gráfica 2.2 guardada
✓ Gráfica 2.2.a guardada
✓ Gráfica 2.2.b guardada
✓ Gráfica 2.2.c guardada

SECCIÓN 3: ANÁLISIS POR DIST
✓ Gráfica 3.1 guardada
✓ Gráfica 3.1.a guardada
✓ Gráfica 3.1.b guardada
✓ Gráfica 3.1.c guardada
✓ Gráfica 3.2 guardada
✓ Gráfica 3.2.a guardada
✓ Gráfica 3.2.b guardada
✓ Gráfica 3.2.c guardada

SECCIÓN 4: ANÁLISIS POR VAR
✓ Gráfica 4.1 guardada
✓ Gráfica 4.1.a guardada
✓ Gráfica 4.1.b guardada
✓ Gráfica 4.1.c guardada
✓ Gráfica 4.2 guardada
✓ Gráfica 4.2.a guardada
✓ Gráfica 4.2.b guardada
✓ Gráfica

# Analisis Diferenciado

## Pre-procesamiento

In [12]:
import pandas as pd
import numpy as np

# Leer los tres archivos
arima_df = pd.read_excel("./datos/resultados_140_ARIMA_FINAL.xlsx")
arima_Diff_df = pd.read_excel("./datos/resultados_140_ARIMA_CON_DIFERENCIACION.xlsx")

# Filtrar los que no tienen "Promedio" en la columna "Paso"
arima_df = arima_df[arima_df['Paso'] != 'Promedio']
arima_Diff_df = arima_Diff_df[arima_Diff_df['Paso'] != 'Promedio']

# Lista de modelos (columnas a promediar)
modelos = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Crear tabla comparativa
comparacion = []

for modelo in modelos:
    fila = {'Modelo': modelo}
    
    # Calcular promedio para cada escenario (de la columna del modelo)
    arima_promedio = arima_df[modelo].mean() if modelo in arima_df.columns else np.nan
    arima_Diff_promedio = arima_Diff_df[modelo].mean() if modelo in arima_Diff_df.columns else np.nan
    
    fila['ARIMA'] = arima_promedio
    fila['ARIMA_Diff'] = arima_Diff_promedio
    
    # Determinar mejor escenario (menor promedio)
    promedios = {
        'ARIMA': arima_promedio,
        'ARIMA_Diff': arima_Diff_promedio
    }
    
    # Filtrar NaN si existen
    promedios_validos = {k: v for k, v in promedios.items() if not pd.isna(v)}
    
    if promedios_validos:
        mejor_escenario = min(promedios_validos, key=promedios_validos.get)
        fila['Mejor_Escenario'] = mejor_escenario
    else:
        fila['Mejor_Escenario'] = 'N/A'
    
    comparacion.append(fila)

# Crear DataFrame con la tabla comparativa
tabla_comparativa = pd.DataFrame(comparacion)

# Redondear valores para mejor visualización
columnas_numericas = ['ARIMA', 'ARIMA_Diff']
tabla_comparativa[columnas_numericas] = tabla_comparativa[columnas_numericas].round(4)

# Mostrar tabla comparativa
print("\n" + "="*80)
print("TABLA COMPARATIVA DE MODELOS POR ESCENARIO")
print("(Promedio de amplitud de intervalos de predicción)")
print("="*80)
print(tabla_comparativa.to_string(index=False))
print("="*80 + "\n")

# Guardar tabla comparativa en Excel
tabla_comparativa.to_excel("Tabla_Comparativa_Modelos_Diff.xlsx", index=False)
print("Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_Diff.xlsx'")

# Agregar columna ESCENARIO a cada DataFrame antes de concatenar
arima_df['ESCENARIO'] = 'Sin diferenciación'
arima_Diff_df['ESCENARIO'] = 'Diferenciado'

# Concatenar los tres dataframes
base_consolidada = pd.concat([arima_df, arima_Diff_df], ignore_index=True)
# Guardar en un archivo Excel
base_consolidada.to_excel("Base_140_diff_escenarios.xlsx", index=False)

print("\nArchivo 'Base_140_diff_escenarios.xlsx' creado exitosamente!")
print(f"\nTotal de filas: {len(base_consolidada)}")
print(f"- ARIMA: {len(arima_df)} filas")
print(f"- ARIMA_Diff: {len(arima_Diff_df)} filas")


TABLA COMPARATIVA DE MODELOS POR ESCENARIO
(Promedio de amplitud de intervalos de predicción)
             Modelo   ARIMA  ARIMA_Diff Mejor_Escenario
              AREPD  9.7604      0.7485      ARIMA_Diff
            AV-MCPS  3.0618      0.6493      ARIMA_Diff
Block Bootstrapping 10.9690      0.6844      ARIMA_Diff
             DeepAR  3.1462      0.5704      ARIMA_Diff
         EnCQR-LSTM  5.8306      0.8656      ARIMA_Diff
               LSPM  1.1140      0.6481      ARIMA_Diff
              LSPMW  3.5094      0.8032      ARIMA_Diff
               MCPS  2.8994      0.6581      ARIMA_Diff
    Sieve Bootstrap  0.5479      0.5454      ARIMA_Diff

Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_Diff.xlsx'

Archivo 'Base_140_diff_escenarios.xlsx' creado exitosamente!

Total de filas: 3360
- ARIMA: 1680 filas
- ARIMA_Diff: 1680 filas


## Analisis general

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import warnings
from matplotlib.patches import Patch

warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./Base_140_diff_escenarios.xlsx"
DIR_SALIDA = "./resultados_escenarios_comparativos"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Colores fijos para barras comparativas
COLOR_SIN_DIFF = '#7f7f7f'  # Gris
COLOR_CON_DIFF = '#1f77b4'  # Azul

# Paleta UNIFICADA para todos los Heatmaps
# RdYlGn_r: Rojo (Valores altos/malos) -> Verde (Valores bajos/buenos)
CMAP_HEATMAP = 'RdYlGn_r' 

# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================

def diebold_mariano_test(errores1, errores2, h=1, alternative='two-sided', loss_function='none'):
    """Test DM Robusto (HAC)"""
    e1 = np.asarray(errores1)
    e2 = np.asarray(errores2)
    n = len(e1)
    
    if loss_function == 'none':
        d = e1 - e2
    elif loss_function == 'squared':
        d = e1**2 - e2**2
    else: # absolute
        d = np.abs(e1) - np.abs(e2)
        
    d_mean = np.mean(d)
    
    if h == 1:
        var_d = np.var(d, ddof=1) / n
    else:
        gamma_0 = np.var(d, ddof=1)
        gamma_sum = 0
        max_lags = min(h-1, n-1)
        for k in range(1, max_lags + 1):
            if k < n:
                gamma_k = np.cov(d[:-k], d[k:], ddof=1)[0,1] if len(d) > k else 0
                gamma_sum += (1 - k/(max_lags+1)) * gamma_k
        var_d = (gamma_0 + 2 * gamma_sum) / n
    
    hlnc = np.sqrt((n + 1 - 2 * h + h * (h - 1) / n) / n) if h > 1 else 1.0
    
    dm_stat = (d_mean / np.sqrt(var_d)) * hlnc if var_d > 0 else 0
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    
    return {'p_value': p_value, 'mean_diff': d_mean, 'dm_statistic': dm_stat}

def comparar_modelo_entre_escenarios(df, modelo, esc_base, esc_diff, alpha=0.05):
    data_base = df[df['ESCENARIO'] == esc_base][modelo].dropna()
    data_diff = df[df['ESCENARIO'] == esc_diff][modelo].dropna()
    
    if len(data_base) < 10 or len(data_diff) < 10:
        return {'Modelo': modelo, 'Conclusión': 'Datos insuficientes', 'p_value': np.nan}
    
    med_base = data_base.median()
    med_diff = data_diff.median()
    mejora_pct = ((med_base - med_diff) / med_base) * 100 if med_base != 0 else 0
    
    try:
        res = diebold_mariano_test(data_base.values, data_diff.values, h=1, loss_function='none')
        sig = res['p_value'] < alpha
        
        if sig:
            conclusion = 'Diferenciación mejora' if res['mean_diff'] > 0 else 'Sin diferenciación es mejor'
        else:
            conclusion = 'Sin diferencia significativa'
            
        return {
            'Modelo': modelo,
            'ECRPS_Sin_Dif': round(med_base, 3),
            'ECRPS_Con_Dif': round(med_diff, 3),
            'Mejora_%': round(mejora_pct, 2),
            'dm_statistic': round(res['dm_statistic'], 4),
            'p_value': res['p_value'],
            'Significativo': 'Sí' if sig else 'No',
            'Conclusión': conclusion
        }
    except:
        return {'Modelo': modelo, 'Conclusión': 'Error'}

# ============================================================================
# CLASE PRINCIPAL
# ============================================================================

class AnalizadorBaseCompleta:

    def __init__(self, ruta_datos):
        print("\n" + "=" * 80)
        print("INICIANDO ANÁLISIS COMPARATIVO (AJUSTADO - BARRAS HORIZONTALES)")
        print("=" * 80 + "\n")

        self.df = pd.read_excel(ruta_datos)
        
        # Limpieza básica
        self.df['ESCENARIO'] = self.df['ESCENARIO'].astype(str).str.strip()
        self.escenarios_unicos = sorted(self.df['ESCENARIO'].unique())
        
        if 'proces_simulacion' in self.df.columns:
            self.df['Tipo de Modelo'] = self.df['proces_simulacion']
        
        self.modelos = [m for m in MODELOS if m in self.df.columns]
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)
        
        # Determinar base vs diff
        if len(self.escenarios_unicos) == 2:
            e1, e2 = self.escenarios_unicos
            if ('sin' in e1.lower() or 'no' in e1.lower()) and not ('sin' in e2.lower()):
                self.esc_base, self.esc_diff = e1, e2
            elif ('sin' in e2.lower() or 'no' in e2.lower()):
                self.esc_base, self.esc_diff = e2, e1
            else:
                self.esc_base, self.esc_diff = e1, e2
        else:
            self.esc_base = self.escenarios_unicos[0]
            self.esc_diff = self.escenarios_unicos[-1]

        print(f"Base: {self.esc_base} | Comparado: {self.esc_diff}")

    def ejecutar_analisis_completo(self):
        print("1️⃣  Comparativa Global...")
        self._1_comparativo_global()
        print("2️⃣  Modelo Generador...")
        self._2_modelo_generador()
        print("3️⃣  Variabilidad IQR (Por Tipo)...")
        self._3_variabilidad_iqr()
        print("4️⃣  Distribución...")
        self._4_distribucion()
        print("5️⃣  Varianza...")
        self._5_varianza_tendencias()
        print("6️⃣  Sensibilidad Ruido...")
        self._6_sensibilidad_ruido()
        print("7️⃣  Robustez (QCD)...")
        self._7_analisis_robustez()
        print("8️⃣  Excel DM...")
        self._analisis_dm_excel()
        print("\n✅ HECHO.")

    # ========================================================================
    # 1. COMPARATIVAS GLOBALES
    # ========================================================================
    def _1_comparativo_global(self):
        datos_agg = self.df.groupby(['ESCENARIO'])[self.modelos].median().T
        
        # 1.1 Barras HORIZONTALES (Izquierda a Derecha)
        fig, ax = plt.subplots(figsize=(15, 10))
        y = np.arange(len(self.modelos))
        height = 0.35 # Altura de la barra en horizontal
        
        # Note: En barh, el primer argumento es Y, el segundo es width (valor X)
        ax.barh(y - height/2, datos_agg[self.esc_base], height, label=self.esc_base, color=COLOR_SIN_DIFF)
        ax.barh(y + height/2, datos_agg[self.esc_diff], height, label=self.esc_diff, color=COLOR_CON_DIFF)

        ax.set_xlabel('Mediana ECRPS', fontweight='bold')
        ax.set_title(f'1.1 Rendimiento: {self.esc_base} vs {self.esc_diff}', fontweight='bold')
        ax.set_yticks(y)
        ax.set_yticklabels(self.modelos)
        ax.invert_yaxis() # Para que el primer modelo esté arriba
        ax.legend()
        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_1_Comparacion_Barras_Horizontales.png', dpi=300)
        plt.close()

        # 1.2 Cambio Porcentual (Ya era Horizontal)
        cambio_pct = ((datos_agg[self.esc_diff] - datos_agg[self.esc_base]) / datos_agg[self.esc_base]) * 100
        cambio_pct = cambio_pct.sort_values()
        
        fig, ax = plt.subplots(figsize=(14, 8))
        norm = plt.Normalize(cambio_pct.min(), cambio_pct.max())
        cmap = plt.get_cmap(CMAP_HEATMAP)
        colores = [cmap(norm(v)) for v in cambio_pct.values]
        
        bars = ax.barh(cambio_pct.index, cambio_pct.values, color=colores, edgecolor='k')
        
        ax.axvline(0, color='k', linestyle='--')
        ax.set_xlabel('Cambio Porcentual del Error (%)', fontweight='bold')
        ax.set_title('1.2 Impacto de la Diferenciación (Verde = Mejora)', fontweight='bold')
        
        for bar in bars:
            w = bar.get_width()
            align = 'left' if w > 0 else 'right'
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.1f}%', va='center', ha=align)

        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_2_Cambio_Porcentual.png', dpi=300)
        plt.close()

    # ========================================================================
    # 2. MODELO GENERADOR
    # ========================================================================
    def _2_modelo_generador(self):
        if 'Tipo de Modelo' not in self.df.columns: return

        df_diff = self.df[self.df['ESCENARIO'] == self.esc_diff]
        df_base = self.df[self.df['ESCENARIO'] == self.esc_base]
        
        piv_diff = df_diff.groupby('Tipo de Modelo')[self.modelos].median()
        piv_base = df_base.groupby('Tipo de Modelo')[self.modelos].median()

        # 2.1 Heatmap Normalizado
        fig, ax = plt.subplots(figsize=(16, 9))
        row_stats = piv_diff.T.agg(['median', 'std'], axis=1)
        zscore = piv_diff.T.sub(row_stats['median'], axis=0).div(row_stats['std'].replace(0,1), axis=0)
        
        sns.heatmap(zscore, annot=True, fmt='.2f', cmap=CMAP_HEATMAP, center=0, ax=ax)
        ax.set_title(f'2.1 Z-Score Rendimiento - {self.esc_diff}', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '2_1_MG_ZScore_Diferenciado.png', dpi=300)
        plt.close()

        # 2.2 Cambio Absoluto
        delta = piv_diff - piv_base
        fig, ax = plt.subplots(figsize=(16, 9))
        sns.heatmap(delta.T, annot=True, fmt='.3f', cmap=CMAP_HEATMAP, center=0, ax=ax)
        ax.set_title(f'2.2 Cambio en ECRPS (Diff - Base)', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '2_2_MG_Cambios.png', dpi=300)
        plt.close()

    # ========================================================================
    # 3. VARIABILIDAD IQR (AGRUPADO POR TIPO)
    # ========================================================================
    def _3_variabilidad_iqr(self):
        if 'Tipo de Modelo' not in self.df.columns: return
        
        # --- PREPARACIÓN DE DATOS PARA 3.1 y 3.2 ---
        # Queremos métricas agregadas por TIPO DE MODELO
        
        tipos_modelo = self.df['Tipo de Modelo'].dropna().unique()
        data_resumen = []

        for tipo in tipos_modelo:
            # Función interna para calcular promedio de IQRs de los modelos en ese tipo
            def calcular_iqr_promedio_tipo(escenario):
                subset = self.df[(self.df['ESCENARIO'] == escenario) & 
                                 (self.df['Tipo de Modelo'] == tipo)]
                if subset.empty: return np.nan
                
                iqrs_individuales = []
                for mod in self.modelos:
                    vals = subset[mod].dropna()
                    if len(vals) > 0:
                        q75, q25 = np.percentile(vals, [75, 25])
                        iqrs_individuales.append(q75 - q25)
                
                return np.mean(iqrs_individuales) if iqrs_individuales else np.nan

            iqr_base = calcular_iqr_promedio_tipo(self.esc_base)
            iqr_diff = calcular_iqr_promedio_tipo(self.esc_diff)
            
            if not np.isnan(iqr_diff):
                delta = iqr_diff - iqr_base if not np.isnan(iqr_base) else np.nan
                data_resumen.append({
                    'Tipo': tipo, 
                    'IQR_Diff': iqr_diff,
                    'Delta_IQR': delta
                })
        
        df_resumen = pd.DataFrame(data_resumen)
        if df_resumen.empty: return

        # 3.1 Gráfico por TIPO DE MODELO - Nivel absoluto (Diff)
        df_31 = df_resumen.sort_values('IQR_Diff')
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.barh(df_31['Tipo'], df_31['IQR_Diff'], color=COLOR_CON_DIFF, alpha=0.8)
        ax.set_title(f'3.1 Variabilidad Promedio (IQR) por TIPO DE MODELO - {self.esc_diff}', fontweight='bold')
        ax.set_xlabel('IQR Promedio del Tipo')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '3_1_IQR_Diferenciado_PorTipo.png', dpi=300)
        plt.close()

        # 3.2 Cambio IQR por TIPO DE MODELO (Delta)
        # Aquí cumplimos el requerimiento: 3.2 ahora es sobre Tipos, no modelos individuales
        df_32 = df_resumen.dropna(subset=['Delta_IQR']).sort_values('Delta_IQR')
        
        fig, ax = plt.subplots(figsize=(12, 8))
        colors = ['green' if x < 0 else 'red' for x in df_32['Delta_IQR']]
        bars = ax.barh(df_32['Tipo'], df_32['Delta_IQR'], color=colors, alpha=0.7, edgecolor='k')
        
        ax.set_title('3.2 Cambio en Variabilidad (Delta IQR) por TIPO DE MODELO\n(Verde = Menos variabilidad con diferenciación)', fontweight='bold')
        ax.axvline(0, color='k', linestyle='--')
        
        for bar in bars:
            w = bar.get_width()
            align = 'left' if w > 0 else 'right'
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.3f}', va='center', ha=align)
            
        plt.tight_layout()
        plt.savefig(self.dir_salida / '3_2_Cambios_IQR_PorTipo.png', dpi=300)
        plt.close()

    # ========================================================================
    # 4. DISTRIBUCIÓN
    # ========================================================================
    def _4_distribucion(self):
        if 'Distribución' not in self.df.columns: return

        piv_diff = self.df[self.df['ESCENARIO'] == self.esc_diff].groupby('Distribución')[self.modelos].median()
        piv_base = self.df[self.df['ESCENARIO'] == self.esc_base].groupby('Distribución')[self.modelos].median()
        
        # 4.1 Heatmap (Diff)
        fig, ax = plt.subplots(figsize=(14, 8))
        sns.heatmap(piv_diff.T, annot=True, fmt='.3f', cmap=CMAP_HEATMAP, ax=ax)
        ax.set_title(f'4.1 Rendimiento por Distribución - {self.esc_diff}', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '4_1_Dist_Heatmap_Diferenciado.png', dpi=300)
        plt.close()

        # 4.2 Heatmap (Delta)
        delta = piv_diff - piv_base
        fig, ax = plt.subplots(figsize=(14, 8))
        sns.heatmap(delta.T, annot=True, fmt='.3f', cmap=CMAP_HEATMAP, center=0, ax=ax)
        ax.set_title(f'4.2 Diferencial (Diff - Base)', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '4_2_Dist_Heatmap_Diferencial.png', dpi=300)
        plt.close()

    # ========================================================================
    # 5. VARIANZA
    # ========================================================================
    def _5_varianza_tendencias(self):
        if 'Varianza error' not in self.df.columns: return
        
        df_diff = self.df[self.df['ESCENARIO'] == self.esc_diff]
        df_base = self.df[self.df['ESCENARIO'] == self.esc_base]

        # 5.1 Tendencias Diferenciado
        fig, ax = plt.subplots(figsize=(14, 8))
        for mod in self.modelos:
            c = df_diff.groupby('Varianza error')[mod].median()
            ax.plot(c.index, c.values, marker='o', label=mod)
        ax.set_title(f'5.1 Sensibilidad a Varianza - {self.esc_diff}', fontweight='bold')
        ax.legend(bbox_to_anchor=(1.01, 1))
        plt.tight_layout()
        plt.savefig(self.dir_salida / '5_1_Varianza_Tendencias_Diferenciado.png', dpi=300)
        plt.close()

        # 5.2 Cambio
        fig, ax = plt.subplots(figsize=(14, 8))
        for mod in self.modelos:
            c1 = df_diff.groupby('Varianza error')[mod].median()
            c2 = df_base.groupby('Varianza error')[mod].median()
            if not c1.empty and not c2.empty:
                delta = c1 - c2
                ax.plot(delta.index, delta.values, marker='o', label=mod)
        ax.axhline(0, color='k', linestyle='--')
        ax.set_title('5.2 Cambio de Comportamiento (Negativo = Mejora)', fontweight='bold')
        ax.legend(bbox_to_anchor=(1.01, 1))
        plt.tight_layout()
        plt.savefig(self.dir_salida / '5_2_Varianza_Cambio_Comportamiento.png', dpi=300)
        plt.close()

    # ========================================================================
    # 6. SENSIBILIDAD RUIDO
    # ========================================================================
    def _6_sensibilidad_ruido(self):
        if 'Varianza error' not in self.df.columns: return

        slopes = []
        for esc in [self.esc_base, self.esc_diff]:
            df_c = self.df[self.df['ESCENARIO'] == esc]
            for mod in self.modelos:
                dat = df_c[['Varianza error', mod]].dropna()
                if len(dat) > 2:
                    res = stats.theilslopes(dat[mod], dat['Varianza error'])
                    slopes.append({'Modelo': mod, 'Esc': esc, 'Slope': res[0]})
        
        df_slopes = pd.DataFrame(slopes)
        if df_slopes.empty: return

        # 6.1 Sensibilidad Diff (Ya era Horizontal)
        sub = df_slopes[df_slopes['Esc'] == self.esc_diff].sort_values('Slope')
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.barh(sub['Modelo'], sub['Slope'], color=COLOR_CON_DIFF)
        ax.set_title(f'6.1 Sensibilidad Ruido (Pendiente) - {self.esc_diff}', fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / '6_1_Sensibilidad_Ruido_Diferenciado.png', dpi=300)
        plt.close()

        # 6.2 Cambio Sensibilidad (Ya era Horizontal)
        piv = df_slopes.pivot(index='Modelo', columns='Esc', values='Slope')
        piv['Delta'] = piv[self.esc_diff] - piv[self.esc_base]
        piv = piv.sort_values('Delta')
        
        fig, ax = plt.subplots(figsize=(12, 8))
        colors = ['green' if x < 0 else 'red' for x in piv['Delta']]
        bars = ax.barh(piv.index, piv['Delta'], color=colors, edgecolor='k')
        ax.axvline(0, color='k')
        ax.set_title('6.2 Cambio en Sensibilidad (Verde = Menos sensible)', fontweight='bold')
        
        for bar in bars:
            w = bar.get_width()
            align = 'left' if w > 0 else 'right'
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.4f}', va='center', ha=align)
            
        plt.tight_layout()
        plt.savefig(self.dir_salida / '6_2_Sensibilidad_Cambio.png', dpi=300)
        plt.close()

    # ========================================================================
    # 7. ROBUSTEZ (QCD) - CON COMPARACIÓN
    # ========================================================================
    def _7_analisis_robustez(self):
        # Calcular QCD para ambos escenarios
        metricas = []
        for esc in [self.esc_base, self.esc_diff]:
            df_c = self.df[self.df['ESCENARIO'] == esc]
            for mod in self.modelos:
                dat = df_c[mod].dropna()
                if len(dat) > 0:
                    q75, q25 = np.percentile(dat, [75, 25])
                    if (q75 + q25) > 0:
                        qcd = (q75 - q25) / (q75 + q25)
                        metricas.append({'Modelo': mod, 'Esc': esc, 'QCD': qcd})
        
        df_qcd = pd.DataFrame(metricas)
        if df_qcd.empty: return

        # 7.1 QCD Diferenciado (Ya era Horizontal)
        sub_diff = df_qcd[df_qcd['Esc'] == self.esc_diff].sort_values('QCD')
        fig, ax = plt.subplots(figsize=(12, 8))
        norm = plt.Normalize(sub_diff['QCD'].min(), sub_diff['QCD'].max())
        cmap = plt.get_cmap(CMAP_HEATMAP)
        colors = [cmap(1 - norm(v)) for v in sub_diff['QCD']]

        bars = ax.barh(sub_diff['Modelo'], sub_diff['QCD'], color=colors, edgecolor='k')
        ax.set_title(f'7.1 Robustez (QCD) - {self.esc_diff} (Menor es mejor)', fontweight='bold')
        
        for bar in bars:
            w = bar.get_width()
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.3f}', va='center', ha='left')
            
        plt.tight_layout()
        plt.savefig(self.dir_salida / '7_1_Robustez_Diferenciado.png', dpi=300)
        plt.close()

        # 7.2 COMPARACIÓN DE ROBUSTEZ (Ya era Horizontal)
        piv = df_qcd.pivot(index='Modelo', columns='Esc', values='QCD')
        piv['Delta'] = piv[self.esc_diff] - piv[self.esc_base]
        piv = piv.sort_values('Delta')
        
        fig, ax = plt.subplots(figsize=(12, 8))
        colors = ['green' if x < 0 else 'red' for x in piv['Delta']]
        bars = ax.barh(piv.index, piv['Delta'], color=colors, edgecolor='k', alpha=0.8)
        
        ax.set_title('7.2 Cambio en Robustez (QCD)\n(Valores negativos indican mayor robustez en diferenciado)', fontweight='bold')
        ax.axvline(0, color='k', linestyle='--')
        
        for bar in bars:
            w = bar.get_width()
            align = 'left' if w > 0 else 'right'
            ax.text(w, bar.get_y() + bar.get_height()/2, f'{w:.3f}', va='center', ha=align)
            
        plt.tight_layout()
        plt.savefig(self.dir_salida / '7_2_Robustez_Cambio.png', dpi=300)
        plt.close()

    # ========================================================================
    # 8. EXCEL
    # ========================================================================
    def _analisis_dm_excel(self):
        res = [comparar_modelo_entre_escenarios(self.df, m, self.esc_base, self.esc_diff) for m in self.modelos]
        df_res = pd.DataFrame(res)
        nombre = self.dir_salida / "Comparacion_DM_Por_Modelo.xlsx"
        df_res.to_excel(nombre, index=False)
        print(f"   ✅ Excel: {nombre}")

# ============================================================================
# MAIN
# ============================================================================
def main():
    try:
        AnalizadorBaseCompleta(RUTA_DATOS).ejecutar_analisis_completo()
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


INICIANDO ANÁLISIS COMPARATIVO (AJUSTADO - BARRAS HORIZONTALES)

Base: Sin diferenciación | Comparado: Diferenciado
1️⃣  Comparativa Global...
2️⃣  Modelo Generador...
3️⃣  Variabilidad IQR (Por Tipo)...
4️⃣  Distribución...
5️⃣  Varianza...
6️⃣  Sensibilidad Ruido...
7️⃣  Robustez (QCD)...
8️⃣  Excel DM...
   ✅ Excel: resultados_escenarios_comparativos\Comparacion_DM_Por_Modelo.xlsx

✅ HECHO.


# Aumento d ARIMA

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from scipy.interpolate import UnivariateSpline
import warnings

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./datos/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx"
DIR_SALIDA = "./resultados_diff_d"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Paleta de colores para modelos
COLORES_MODELOS = plt.cm.tab10(np.linspace(0, 1, len(MODELOS)))
COLOR_MAP_MODELOS = {mod: COLORES_MODELOS[i] for i, mod in enumerate(MODELOS)}

# Colores para modalidades
COLOR_SIN_DIFF = '#e74c3c'  # Rojo
COLOR_CON_DIFF = '#3498db'  # Azul

CMAP_HEATMAP = 'RdYlGn_r' # Rojo = Malo (Alto Error), Verde = Bueno (Bajo Error)

# ============================================================================
# CLASE PRINCIPAL
# ============================================================================

class AnalizadorSensibilidadD:
    
    def __init__(self, ruta_datos):
        print("\n" + "=" * 80)
        print("ANÁLISIS DE SENSIBILIDAD AL PARÁMETRO D (ORDEN DE DIFERENCIACIÓN)")
        print("=" * 80 + "\n")
        
        self.df = pd.read_excel(ruta_datos)
        
        # Limpieza
        self.df['Modalidad'] = self.df['Modalidad'].astype(str).str.strip().str.upper()
        self.df['d'] = pd.to_numeric(self.df['d'], errors='coerce')
        
        # Filtrar datos válidos
        self.df = self.df[self.df['d'].notna()].copy()
        self.valores_d = sorted(self.df['d'].unique())
        
        self.modelos = [m for m in MODELOS if m in self.df.columns]
        self.modalidades = sorted(self.df['Modalidad'].unique())
        
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)
        
        print(f"📊 Valores de d: {self.valores_d}")
        print(f"🎭 Modalidades: {self.modalidades}")
        print(f"📈 Modelos analizados: {len(self.modelos)}\n")

    def _fmt(self, x):
        """
        Formatea números: 
        - Usa notación científica si abs(x) >= 10,000 (5 dígitos) o x < 0.001
        - Usa decimales estándar en caso contrario.
        """
        if pd.isna(x):
            return ""
        if x == 0:
            return "0"
        # Umbral: 5 dígitos enteros (10,000) o muy pequeños
        if abs(x) >= 10000 or (0 < abs(x) < 0.001):
            return f"{x:.2e}"
        return f"{x:.3f}"

    def ejecutar_analisis_completo(self):
        """Ejecuta todas las preguntas de investigación"""
        
        # --- VISUALIZACIÓN GENERAL ---
        self._visualizar_heatmaps_ecrps_absolutos()

        print("\n" + "=" * 80)
        print("PREGUNTA 1: ¿Qué modelo es más sensible a los cambios en d?")
        print("=" * 80)
        self._pregunta1_sensibilidad_modelos()
        
        print("\n" + "=" * 80)
        print("PREGUNTA 2: ¿Existe un punto de inflexión en d?")
        print("=" * 80)
        self._pregunta2_punto_inflexion()
        
        print("\n" + "=" * 80)
        print("PREGUNTA 3: ¿Cómo impacta d en la variabilidad?")
        print("=" * 80)
        self._pregunta3_variabilidad()
        
        print("\n" + "=" * 80)
        print("PREGUNTA 4: ¿La diferenciación previa amplifica el efecto de d?")
        print("=" * 80)
        self._pregunta4_interaccion_modalidad()
        
        print("\n" + "=" * 80)
        print("PREGUNTA 5: ¿Cuándo es significativa la diferenciación? (Foco: Sieve Bootstrap)")
        print("=" * 80)
        self._pregunta5_consistencia()
        
        print("\n✅ ANÁLISIS COMPLETO FINALIZADO")

    # ========================================================================
    # VISUALIZACIÓN EXTRA: HEATMAPS DE ECRPS MEDIO (Modelo vs d)
    # ========================================================================
    def _visualizar_heatmaps_ecrps_absolutos(self):
        print("📊 Generando Heatmaps Generales de Rendimiento...")

        fig, axes = plt.subplots(1, 2, figsize=(24, 10))
        
        vmin = self.df[self.modelos].min().min()
        vmax_robust = np.percentile(self.df[self.modelos].values, 95)

        for idx, modalidad in enumerate(self.modalidades):
            if idx >= 2: break 
            
            ax = axes[idx]
            df_mod = self.df[self.df['Modalidad'] == modalidad]
            
            heatmap_data = df_mod.groupby('d')[self.modelos].mean().T
            
            # Crear matriz de anotaciones formateadas
            annot_data = heatmap_data.applymap(self._fmt)
            
            sns.heatmap(heatmap_data, ax=ax, cmap=CMAP_HEATMAP, 
                        annot=annot_data.values, fmt='', # fmt='' es necesario cuando pasamos strings
                        vmin=vmin, vmax=vmax_robust, 
                        linewidths=.5, cbar_kws={'label': 'ECRPS Medio'})
            
            ax.set_title(f'Rendimiento Medio (ECRPS) - {modalidad}', fontweight='bold', fontsize=14)
            ax.set_xlabel('Orden de Diferenciación (d)', fontsize=12, fontweight='bold')
            ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
            ax.tick_params(axis='y', rotation=0)

        plt.suptitle('Comparación de ECRPS Medio: Modelos vs. Orden de Diferenciación (d)', 
                     fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        
        nombre_archivo = "0_General_Heatmaps_ECRPS_Medio.png"
        plt.savefig(self.dir_salida / nombre_archivo, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅ Gráfico guardado: {nombre_archivo}")

    # ========================================================================
    # PREGUNTA 1: SENSIBILIDAD DE MODELOS A d
    # ========================================================================
    def _pregunta1_sensibilidad_modelos(self):
        resultados = []
        for modalidad in self.modalidades:
            df_mod = self.df[self.df['Modalidad'] == modalidad]
            for modelo in self.modelos:
                serie = df_mod.groupby('d')[modelo].median()
                if len(serie) > 3:
                    slope, intercept, _, _ = stats.theilslopes(serie.values, serie.index)
                    corr, p_value = stats.spearmanr(serie.index, serie.values)
                    rango = serie.max() - serie.min()
                    variacion_pct = (rango / serie.mean()) * 100 if serie.mean() != 0 else 0
                    
                    resultados.append({
                        'Modelo': modelo, 'Modalidad': modalidad, 'Pendiente': slope,
                        'Correlación': corr, 'p_value': p_value, 'Rango': rango,
                        'Variación_%': variacion_pct, 'Sensibilidad_Score': abs(slope) * abs(corr)
                    })
        
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P1_Sensibilidad_Modelos.xlsx", index=False)
        print(f"   ✅ Excel guardado: P1_Sensibilidad_Modelos.xlsx")
        
        self._visualizar_pregunta1(df_resultados)
        
        print("\n🔥 TOP 3 MODELOS MÁS SENSIBLES A d:")
        top_sensibles = df_resultados.nlargest(3, 'Sensibilidad_Score')
        for idx, row in top_sensibles.iterrows():
            print(f"   {row['Modelo']} ({row['Modalidad']}): Score={self._fmt(row['Sensibilidad_Score'])}")

    def _visualizar_pregunta1(self, df_resultados):
        # 1.1 Ranking
        fig, axes = plt.subplots(1, 2, figsize=(18, 8))
        for idx, modalidad in enumerate(self.modalidades):
            if idx >= 2: break
            df_sub = df_resultados[df_resultados['Modalidad'] == modalidad].sort_values('Sensibilidad_Score')
            ax = axes[idx]
            colors = [COLOR_MAP_MODELOS[m] for m in df_sub['Modelo']]
            bars = ax.barh(df_sub['Modelo'], df_sub['Sensibilidad_Score'], color=colors, edgecolor='black', alpha=0.8)
            ax.set_xlabel('Score de Sensibilidad', fontweight='bold')
            ax.set_title(f'P1.1: Sensibilidad a d - {modalidad}', fontweight='bold')
            ax.grid(axis='x', alpha=0.3)
            
            # Anotación formateada
            for bar in bars:
                width = bar.get_width()
                label = self._fmt(width)
                ax.text(width, bar.get_y() + bar.get_height()/2, label, va='center', fontsize=9)

        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P1_1_Ranking_Sensibilidad.png', dpi=300)
        plt.close()
        
        # 1.2 Comparación
        fig, ax = plt.subplots(figsize=(14, 8))
        piv = df_resultados.pivot(index='Modelo', columns='Modalidad', values='Pendiente')
        piv.plot(kind='bar', ax=ax, width=0.7, edgecolor='black', 
                 color=[COLOR_SIN_DIFF, COLOR_CON_DIFF] if len(piv.columns)==2 else None)
        ax.set_ylabel('Pendiente (ECRPS vs d)')
        ax.set_title('P1.2: Pendientes de Sensibilidad por Modalidad')
        ax.axhline(0, color='black', linestyle='--')
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P1_2_Pendientes_Comparacion.png', dpi=300)
        plt.close()

    # ========================================================================
    # PREGUNTA 2: PUNTO DE INFLEXIÓN
    # ========================================================================
    def _pregunta2_punto_inflexion(self):
        resultados = []
        for modalidad in self.modalidades:
            df_mod = self.df[self.df['Modalidad'] == modalidad]
            for modelo in self.modelos:
                serie = df_mod.groupby('d')[modelo].median().dropna()
                if len(serie) >= 5:
                    x, y = serie.index.values, serie.values
                    try:
                        spline = UnivariateSpline(x, y, s=0.1, k=3)
                        x_fine = np.linspace(x.min(), x.max(), 100)
                        y_second_deriv = spline.derivative(n=2)(x_fine)
                        d_inflexion = x_fine[np.argmax(np.abs(y_second_deriv))]
                        resultados.append({
                            'Modelo': modelo, 'Modalidad': modalidad, 'd_Inflexión': round(d_inflexion, 1),
                            'Cambio_Total_%': ((serie.iloc[-1] - serie.iloc[0]) / serie.iloc[0] * 100) if serie.iloc[0] != 0 else 0
                        })
                    except: pass
        
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P2_Puntos_Inflexion.xlsx", index=False)
        print(f"   ✅ Excel guardado: P2_Puntos_Inflexion.xlsx")
        self._visualizar_pregunta2(df_resultados)

    def _visualizar_pregunta2(self, df_resultados):
        fig, ax = plt.subplots(figsize=(14, 8))
        for modalidad in self.modalidades:
            df_sub = df_resultados[df_resultados['Modalidad'] == modalidad]
            color = COLOR_SIN_DIFF if 'SIN' in modalidad else COLOR_CON_DIFF
            ax.scatter(df_sub['d_Inflexión'], df_sub['Modelo'], s=200, alpha=0.7, color=color, edgecolor='black', label=modalidad, marker='D')
        ax.axvline(df_resultados['d_Inflexión'].mean(), color='gray', linestyle='--', label='Media Global')
        ax.set_xlabel('Valor de d en Punto de Inflexión')
        ax.set_title('P2.1: Distribución de Puntos de Inflexión')
        ax.legend()
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P2_1_Distribucion_Inflexion.png', dpi=300)
        plt.close()

    # ========================================================================
    # PREGUNTA 3: IMPACTO EN VARIABILIDAD
    # ========================================================================
    def _pregunta3_variabilidad(self):
        resultados = []
        for modalidad in self.modalidades:
            df_mod = self.df[self.df['Modalidad'] == modalidad]
            for d_val in self.valores_d:
                df_d = df_mod[df_mod['d'] == d_val]
                for modelo in self.modelos:
                    datos = df_d[modelo].dropna()
                    if len(datos) > 5:
                        q75, q25 = np.percentile(datos, [75, 25])
                        resultados.append({
                            'Modelo': modelo, 'Modalidad': modalidad, 'd': d_val,
                            'IQR': q75 - q25,
                            'QCD': (q75 - q25) / (q75 + q25) if (q75 + q25) > 0 else 0
                        })
        
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P3_Variabilidad_por_d.xlsx", index=False)
        print(f"   ✅ Excel guardado: P3_Variabilidad_por_d.xlsx")
        self._visualizar_pregunta3(df_resultados)

    def _visualizar_pregunta3(self, df_resultados):
        fig, axes = plt.subplots(1, 2, figsize=(18, 7))
        for idx, modalidad in enumerate(self.modalidades):
            if idx >= 2: break
            df_mod = df_resultados[df_resultados['Modalidad'] == modalidad]
            ax = axes[idx]
            for modelo in self.modelos:
                df_m = df_mod[df_mod['Modelo'] == modelo]
                if not df_m.empty:
                    ax.plot(df_m['d'], df_m['IQR'], marker='o', label=modelo, color=COLOR_MAP_MODELOS[modelo])
            ax.set_title(f'P3.1: Evolución IQR - {modalidad}')
            ax.set_ylabel('IQR')
            ax.set_xlabel('d')
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P3_1_Evolucion_IQR.png', dpi=300)
        plt.close()
        
        # Heatmap QCD
        for modalidad in self.modalidades:
            piv = df_resultados[df_resultados['Modalidad'] == modalidad].pivot(index='Modelo', columns='d', values='QCD')
            
            # Formatear anotaciones
            annot_qcd = piv.applymap(self._fmt)
            
            fig, ax = plt.subplots(figsize=(12, 6))
            sns.heatmap(piv, annot=annot_qcd.values, fmt='', cmap=CMAP_HEATMAP, ax=ax)
            ax.set_title(f'P3.2: Robustez Relativa (QCD) - {modalidad}')
            plt.tight_layout()
            plt.savefig(self.dir_salida / f'P3_2_Heatmap_QCD_{modalidad}.png', dpi=300)
            plt.close()

    # ========================================================================
    # PREGUNTA 4: INTERACCIÓN MODALIDAD × d
    # ========================================================================
    def _pregunta4_interaccion_modalidad(self):
        if len(self.modalidades) < 2: return
        resultados = []
        for modelo in self.modelos:
            pendientes = {}
            for modalidad in self.modalidades:
                df_mod = self.df[self.df['Modalidad'] == modalidad]
                serie = df_mod.groupby('d')[modelo].median()
                if len(serie) > 3:
                    slope, _, _, _ = stats.theilslopes(serie.values, serie.index)
                    pendientes[modalidad] = slope
            
            if len(pendientes) == 2:
                mod1, mod2 = self.modalidades
                interaccion = pendientes[mod2] - pendientes[mod1]
                resultados.append({
                    'Modelo': modelo, 'Pendiente_Diff': pendientes.get(mod2,0), 'Pendiente_Base': pendientes.get(mod1,0),
                    'Interacción': interaccion,
                    'Interpretación': 'Amplifica' if abs(pendientes[mod2]) > abs(pendientes[mod1]) else 'Modera'
                })
        
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P4_Interaccion_Modalidad.xlsx", index=False)
        print(f"   ✅ Excel guardado: P4_Interaccion_Modalidad.xlsx")
        self._visualizar_pregunta4(df_resultados)

    def _visualizar_pregunta4(self, df_resultados):
        df_sorted = df_resultados.sort_values('Interacción')
        fig, ax = plt.subplots(figsize=(14, 8))
        colors = ['#27ae60' if x < 0 else '#e74c3c' for x in df_sorted['Interacción']]
        bars = ax.barh(df_sorted['Modelo'], df_sorted['Interacción'], color=colors, edgecolor='black', alpha=0.8)
        ax.axvline(0, color='black', linestyle='--')
        ax.set_xlabel('Efecto de Interacción (Pendiente_Diff - Pendiente_Base)')
        ax.set_title('P4.2: Efecto de la Diferenciación sobre Sensibilidad a d')
        
        for bar in bars:
            width = bar.get_width()
            label = self._fmt(width)
            ax.text(width, bar.get_y() + bar.get_height()/2, label, 
                   ha='left' if width > 0 else 'right', va='center', fontsize=9, fontweight='bold')
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P4_2_Efecto_Interaccion.png', dpi=300)
        plt.close()

    # ========================================================================
    # PREGUNTA 5: SIGNIFICANCIA DE LA DIFERENCIACIÓN (SIEVE BOOTSTRAP)
    # ========================================================================
    def _pregunta5_consistencia(self):
        print(f"\n🔎 Analizando significancia estadística (Mann-Whitney U)...")
        resultados = []
        if len(self.modalidades) < 2:
            print("   ⚠️  No es posible comparar (falta modalidad).")
            return

        mod_sin = [m for m in self.modalidades if 'SIN' in m][0]
        mod_con = [m for m in self.modalidades if 'CON' in m][0]

        for modelo in self.modelos:
            for d_val in self.valores_d:
                datos_sin = self.df[(self.df['Modalidad'] == mod_sin) & (self.df['d'] == d_val)][modelo].dropna()
                datos_con = self.df[(self.df['Modalidad'] == mod_con) & (self.df['d'] == d_val)][modelo].dropna()
                
                if len(datos_sin) > 3 and len(datos_con) > 3:
                    stat, p_value = stats.mannwhitneyu(datos_sin, datos_con, alternative='two-sided')
                    diff_mediana = datos_sin.median() - datos_con.median()
                    resultados.append({
                        'Modelo': modelo, 'd': d_val, 'p_value': p_value,
                        'Significativo': p_value < 0.05, 'Diff_Mediana': diff_mediana
                    })

        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_excel(self.dir_salida / "P5_Significancia_Diferenciacion.xlsx", index=False)
        print(f"   ✅ Excel guardado: P5_Significancia_Diferenciacion.xlsx")
        
        self._visualizar_pregunta5(df_resultados)
        
        print("\n🧐 ANÁLISIS ESPECÍFICO: Sieve Bootstrap")
        df_sb = df_resultados[df_resultados['Modelo'] == 'Sieve Bootstrap'].sort_values('d')
        if not df_sb.empty:
            significativos = df_sb[df_sb['Significativo']]
            if not significativos.empty:
                primer_d = significativos['d'].iloc[0]
                print(f"   👉 Para Sieve Bootstrap, la diferenciación es estadísticamente significativa (p < 0.05)")
                print(f"      a partir de d = {primer_d}")
            else:
                print("   👉 No se encontraron diferencias significativas para Sieve Bootstrap en ningún d.")
        else:
            print("   ⚠️  Sieve Bootstrap no encontrado en los resultados.")

    def _visualizar_pregunta5(self, df_resultados):
        piv_p = df_resultados.pivot(index='Modelo', columns='d', values='p_value')
        log_p = -np.log10(piv_p + 1e-10) 
        
        # Formatear P-values para anotación
        annot_p = piv_p.applymap(self._fmt)

        fig, ax = plt.subplots(figsize=(14, 8))
        # Usamos 'magma' en minúsculas para evitar KeyError
        sns.heatmap(log_p, cmap='magma', annot=annot_p.values, fmt='', 
                    cbar_kws={'label': '-log10(p-value)'}, ax=ax)
        
        ax.set_title('P5.1: Significancia Estadística (p-values) - Valores claros = Muy Significativo')
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P5_1_Heatmap_Significancia.png', dpi=300)
        plt.close()
        
        # 5.2 Evolución P-value
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.axhline(0.05, color='red', linestyle='--', label='p=0.05')
        
        df_sb = df_resultados[df_resultados['Modelo'] == 'Sieve Bootstrap']
        if not df_sb.empty:
            ax.plot(df_sb['d'], df_sb['p_value'], marker='o', linewidth=3, color='#8e44ad', label='Sieve Bootstrap')
        
        for modelo in self.modelos:
            if modelo != 'Sieve Bootstrap':
                df_m = df_resultados[df_resultados['Modelo'] == modelo]
                ax.plot(df_m['d'], df_m['p_value'], color='gray', alpha=0.15)
        
        ax.set_yscale('log')
        ax.set_ylabel('P-value (Log)')
        ax.set_title('P5.2: Evolución de P-value para Sieve Bootstrap')
        ax.legend()
        plt.tight_layout()
        plt.savefig(self.dir_salida / 'P5_2_Sieve_Bootstrap_Significancia.png', dpi=300)
        plt.close()

# ============================================================================
# EJECUCIÓN
# ============================================================================

if __name__ == "__main__":
    if Path(RUTA_DATOS).exists():
        analizador = AnalizadorSensibilidadD(RUTA_DATOS)
        analizador.ejecutar_analisis_completo()
    else:
        print(f"\n⛔ ERROR: No se encontró el archivo: {RUTA_DATOS}")


ANÁLISIS DE SENSIBILIDAD AL PARÁMETRO D (ORDEN DE DIFERENCIACIÓN)

📊 Valores de d: [np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(7), np.int64(10)]
🎭 Modalidades: ['CON_DIFF', 'SIN_DIFF']
📈 Modelos analizados: 9

📊 Generando Heatmaps Generales de Rendimiento...
   ✅ Gráfico guardado: 0_General_Heatmaps_ECRPS_Medio.png

PREGUNTA 1: ¿Qué modelo es más sensible a los cambios en d?
   ✅ Excel guardado: P1_Sensibilidad_Modelos.xlsx

🔥 TOP 3 MODELOS MÁS SENSIBLES A d:
   Block Bootstrapping (SIN_DIFF): Score=8.23e+10
   AREPD (SIN_DIFF): Score=8.04e+10
   DeepAR (SIN_DIFF): Score=7.69e+10

PREGUNTA 2: ¿Existe un punto de inflexión en d?
   ✅ Excel guardado: P2_Puntos_Inflexion.xlsx

PREGUNTA 3: ¿Cómo impacta d en la variabilidad?
   ✅ Excel guardado: P3_Variabilidad_por_d.xlsx

PREGUNTA 4: ¿La diferenciación previa amplifica el efecto de d?
   ✅ Excel guardado: P4_Interaccion_Modalidad.xlsx

PREGUNTA 5: ¿Cuándo es significativa la diferenciación? (Foco: Sieve Bootstrap)

🔎 An

# Analisis cambio de entrenamiento

## Pre-procesamiento

In [15]:
import pandas as pd
import numpy as np

# Leer los tres archivos
arma_df = pd.read_excel("./datos/resultados_TAMANOS_CRECIENTES_ARMA.xlsx")
arima_df = pd.read_excel("./datos/resultados_TAMANOS_CRECIENTES_ARIMA.xlsx")
setar_df = pd.read_excel("./datos/resultados_TAMANOS_CRECIENTES_SETAR.xlsx")

# Filtrar los que no tienen "Promedio" en la columna "Paso"
arma_df = arma_df[arma_df['Paso'] != 'Promedio']
arima_df = arima_df[arima_df['Paso'] != 'Promedio']
setar_df = setar_df[setar_df['Paso'] != 'Promedio']

# Lista de modelos (columnas a promediar)
modelos = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Crear tabla comparativa
comparacion = []

for modelo in modelos:
    fila = {'Modelo': modelo}
    
    # Calcular promedio para cada escenario (de la columna del modelo)
    arma_promedio = arma_df[modelo].mean() if modelo in arma_df.columns else np.nan
    arima_promedio = arima_df[modelo].mean() if modelo in arima_df.columns else np.nan
    setar_promedio = setar_df[modelo].mean() if modelo in setar_df.columns else np.nan
    
    fila['ARMA'] = arma_promedio
    fila['ARIMA'] = arima_promedio
    fila['SETAR'] = setar_promedio
    
    # Determinar mejor escenario (menor promedio)
    promedios = {
        'ARMA': arma_promedio,
        'ARIMA': arima_promedio,
        'SETAR': setar_promedio
    }
    
    # Filtrar NaN si existen
    promedios_validos = {k: v for k, v in promedios.items() if not pd.isna(v)}
    
    if promedios_validos:
        mejor_escenario = min(promedios_validos, key=promedios_validos.get)
        fila['Mejor_Escenario'] = mejor_escenario
    else:
        fila['Mejor_Escenario'] = 'N/A'
    
    comparacion.append(fila)

# Crear DataFrame con la tabla comparativa
tabla_comparativa = pd.DataFrame(comparacion)

# Redondear valores para mejor visualización
columnas_numericas = ['ARMA', 'ARIMA', 'SETAR']
tabla_comparativa[columnas_numericas] = tabla_comparativa[columnas_numericas].round(4)

# Mostrar tabla comparativa
print("\n" + "="*80)
print("TABLA COMPARATIVA DE MODELOS POR ESCENARIO")
print("(Promedio de amplitud de intervalos de predicción)")
print("="*80)
print(tabla_comparativa.to_string(index=False))
print("="*80 + "\n")

# Guardar tabla comparativa en Excel
tabla_comparativa.to_excel("Tabla_Comparativa_Modelos_tamaño.xlsx", index=False)
print("Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_tamaño.xlsx'")

# Procesamiento especial para SETAR
if 'Descripción' in setar_df.columns:
    setar_df = setar_df.drop('Descripción', axis=1)

# Agregar columna ESCENARIO a cada DataFrame antes de concatenar
arma_df['ESCENARIO'] = 'Lineal - estacionario'
arima_df['ESCENARIO'] = 'Lineal - NO estacionario'
setar_df['ESCENARIO'] = 'NO lineal - estacionario'

# Concatenar los tres dataframes
base_consolidada = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Guardar en un archivo Excel
base_consolidada.to_excel("Base_Tamaño_3_escenarios.xlsx", index=False)

print("\nArchivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!")
print(f"\nTotal de filas: {len(base_consolidada)}")
print(f"- ARMA: {len(arma_df)} filas")
print(f"- ARIMA: {len(arima_df)} filas")
print(f"- SETAR: {len(setar_df)} filas")


TABLA COMPARATIVA DE MODELOS POR ESCENARIO
(Promedio de amplitud de intervalos de predicción)
             Modelo   ARMA   ARIMA  SETAR Mejor_Escenario
              AREPD 0.9345 13.0489 0.6971           SETAR
            AV-MCPS 0.6768  3.5050 0.6531           SETAR
Block Bootstrapping 0.9049 15.1183 0.6318           SETAR
             DeepAR 0.5650  3.6998 0.5845            ARMA
         EnCQR-LSTM 0.9515  5.9330 0.8404           SETAR
               LSPM 0.7689  1.0868 0.6586           SETAR
              LSPMW 0.7931  1.0870 0.6754           SETAR
               MCPS 0.6496  3.2780 0.6325           SETAR
    Sieve Bootstrap 0.5541  0.5583 0.6254            ARMA

Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_tamaño.xlsx'

Archivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!

Total de filas: 126000
- ARMA: 42000 filas
- ARIMA: 42000 filas
- SETAR: 42000 filas


## Analisis general

In [16]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from itertools import combinations
import warnings
import gc

# Ignorar advertencias
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Crear carpeta de resultados
output_dir = Path("./resultados_analisis_ntotal")
output_dir.mkdir(parents=True, exist_ok=True)

# Configuración
archivo_excel = "./Base_Tamaño_3_escenarios.xlsx"
MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Cargar datos
print("Cargando datos...")
try:
    df = pd.read_excel(archivo_excel)
except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo '{archivo_excel}'.")
    exit()

print(f"Columnas disponibles: {df.columns.tolist()}")
print(f"N_Total únicos: {sorted(df['N_Total'].unique())}")
print(f"Número de observaciones: {len(df)}")

# ============================================================================
# FUNCIÓN DIEBOLD-MARIANO
# ============================================================================

def diebold_mariano_test(series1, series2):
    """
    Test de Diebold-Mariano.
    Asumimos que las series ingresadas ya son las pérdidas (ECRPS), 
    por lo que comparamos d = L1 - L2 directamente o cuadrática según se desee.
    Aquí se mantiene la lógica cuadrática estándar del test sobre la diferencia.
    """
    d = series1**2 - series2**2
    d = d.dropna()
    
    if len(d) < 2:
        return np.nan, np.nan
    
    d_mean = d.mean()
    n = len(d)
    d_var = d.var() / n
    
    if d_var <= 0:
        return np.nan, np.nan
    
    dm_stat = d_mean / np.sqrt(d_var)
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    
    return dm_stat, p_value

# ============================================================================
# ANÁLISIS 1: HEATMAPS - PROMEDIO DE ECRPS POR TIPO_PROCESO Y N_TOTAL
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 1: HEATMAPS - Promedio de ECRPS por Proceso y N_Total")
print("="*80)

for modelo in MODELOS:
    # Calcular promedio de ECRPS (asumiendo que el valor en excel ya es el score positivo)
    pivot_data = df.groupby(['Proceso', 'N_Total'])[modelo].mean().reset_index()
    pivot_table = pivot_data.pivot(index='Proceso', columns='N_Total', values=modelo)
    
    # Crear heatmap
    fig, ax = plt.subplots(figsize=(14, 6))
    
    sns.heatmap(pivot_table, 
                annot=True, 
                fmt='.2f',  # <--- CAMBIO: Solo 2 dígitos
                cmap='RdYlGn_r',  # Rojo=alto ECRPS (malo), Verde=bajo ECRPS (bueno)
                cbar_kws={'label': 'ECRPS Promedio', 'shrink': 0.7}, # Barra reducida al 70%
                linewidths=0.5,
                linecolor='gray',
                ax=ax)
    
    plt.title(f'Heatmap: {modelo}\nECRPS Promedio por Proceso y N_Total',
              fontsize=14, fontweight='bold', pad=20)
    plt.xlabel('N_Total (Tamaño de Muestra)', fontsize=11)
    plt.ylabel('Tipo de Proceso', fontsize=11)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    archivo_heatmap = output_dir / f"heatmap_{modelo.replace(' ', '_')}.png"
    plt.savefig(archivo_heatmap, dpi=300, bbox_inches='tight')
    print(f"Heatmap guardado: {archivo_heatmap}")
    plt.close()

# ============================================================================
# ANÁLISIS 2: MEJOR N_TOTAL POR TIPO_PROCESO Y MODELO
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 2: MEJOR N_TOTAL (Menor ECRPS Promedio)")
print("="*80)

resultados_mejor_ntotal = []

for tipo_proceso in df['Tipo_Proceso'].unique():
    for modelo in MODELOS:
        df_filtrado = df[df['Tipo_Proceso'] == tipo_proceso]
        
        # Calcular ECRPS promedio por N_Total
        promedios = df_filtrado.groupby('N_Total')[modelo].mean()
        
        mejor_ntotal = promedios.idxmin()
        mejor_score = promedios.min()
        
        resultados_mejor_ntotal.append({
            'Tipo_Proceso': tipo_proceso,
            'Modelo': modelo,
            'Mejor_N_Total': mejor_ntotal,
            'ECRPS_Promedio': round(mejor_score, 4)
        })
        
        print(f"{tipo_proceso} | {modelo}: N_Total={mejor_ntotal} (ECRPS={mejor_score:.4f})")

# Guardar en Excel
df_mejor_ntotal = pd.DataFrame(resultados_mejor_ntotal)
archivo_mejor = output_dir / "mejor_ntotal_por_modelo_y_tipo.xlsx"
df_mejor_ntotal.to_excel(archivo_mejor, index=False)

# ============================================================================
# ANÁLISIS 3: COMPARACIÓN ENTRE N_TOTALES CON DIEBOLD-MARIANO
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 3: Comparaciones DM entre N_Totales")
print("="*80)

resultados_dm = []

for tipo_proceso in df['Tipo_Proceso'].unique():
    for modelo in MODELOS:
        df_filtrado = df[df['Tipo_Proceso'] == tipo_proceso]
        ntotales_disponibles = sorted(df_filtrado['N_Total'].unique())
        
        for ntotal1, ntotal2 in combinations(ntotales_disponibles, 2):
            series1 = df_filtrado[df_filtrado['N_Total'] == ntotal1][modelo]
            series2 = df_filtrado[df_filtrado['N_Total'] == ntotal2][modelo]
            
            if len(series1) > 0 and len(series2) > 0:
                dm_stat, p_value = diebold_mariano_test(series1, series2)
                
                mean1 = series1.mean()
                mean2 = series2.mean()
                
                es_significativo = p_value < 0.05 if not np.isnan(p_value) else False
                
                if es_significativo:
                    if mean1 < mean2:
                        ganador = ntotal1
                        diferencia = "N_Total={} es significativamente MEJOR".format(ntotal1)
                    else:
                        ganador = ntotal2
                        diferencia = "N_Total={} es significativamente MEJOR".format(ntotal2)
                else:
                    ganador = None
                    diferencia = "Sin diferencia significativa"
                
                resultados_dm.append({
                    'Tipo_Proceso': tipo_proceso,
                    'Modelo': modelo,
                    'N_Total_1': ntotal1,
                    'N_Total_2': ntotal2,
                    'ECRPS_1': round(mean1, 4),
                    'ECRPS_2': round(mean2, 4),
                    'P_Value': round(p_value, 4) if not np.isnan(p_value) else None,
                    'Significativo': 'Sí' if es_significativo else 'No',
                    'Interpretación': diferencia
                })

df_dm = pd.DataFrame(resultados_dm)
archivo_dm = output_dir / "comparaciones_ntotal_diebold_mariano.xlsx"
with pd.ExcelWriter(archivo_dm, engine='openpyxl') as writer:
    df_dm.to_excel(writer, sheet_name='Resultados', index=False)

# ============================================================================
# ANÁLISIS 4: RESUMEN ESTADÍSTICO
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 4: Resumen Estadístico (ECRPS)")
print("="*80)

resumen_stats = []
for tipo_proceso in df['Tipo_Proceso'].unique():
    for ntotal in sorted(df['N_Total'].unique()):
        df_filtrado = df[(df['Tipo_Proceso'] == tipo_proceso) & (df['N_Total'] == ntotal)]
        
        for modelo in MODELOS:
            vals = df_filtrado[modelo] # Asumimos ECRPS directo
            resumen_stats.append({
                'Tipo_Proceso': tipo_proceso,
                'N_Total': ntotal,
                'Modelo': modelo,
                'Media': round(vals.mean(), 4),
                'Mediana': round(vals.median(), 4),
                'Desv_Std': round(vals.std(), 4),
                'Min': round(vals.min(), 4),
                'Max': round(vals.max(), 4)
            })

df_resumen = pd.DataFrame(resumen_stats)
df_resumen.to_excel(output_dir / "resumen_estadistico_ntotal.xlsx", index=False)

# ============================================================================
# ANÁLISIS 5: GRÁFICO DE LÍNEAS - EVOLUCIÓN POR TIPO DE PROCESO
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 5: Evolución del ECRPS por Proceso")
print("="*80)

for tipo_proceso in df['Tipo_Proceso'].unique():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    for modelo in MODELOS:
        df_filtrado = df[df['Tipo_Proceso'] == tipo_proceso]
        promedios = df_filtrado.groupby('N_Total')[modelo].mean()
        
        ax.plot(promedios.index, promedios.values, marker='o', linewidth=2, label=modelo)
    
    ax.set_xlabel('N_Total (Tamaño de Muestra)', fontsize=12)
    ax.set_ylabel('ECRPS Promedio', fontsize=12)
    ax.set_title(f'Evolución del ECRPS por N_Total\nTipo de Proceso: {tipo_proceso}',
                 fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10, bbox_to_anchor=(1.15, 1))
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    
    archivo_lineas = output_dir / f"evolucion_ecrps_{tipo_proceso.replace(' ', '_')}.png"
    plt.savefig(archivo_lineas, dpi=300, bbox_inches='tight')
    plt.close()

# ============================================================================
# ANÁLISIS 5B: TENDENCIA GLOBAL (TODOS LOS MODELOS EN UNA GRÁFICA)
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 5B: Tendencia Global de Modelos (ECRPS vs N_Total)")
print("="*80)

# Agrupar por N_Total para todos los procesos juntos (Promedio General)
fig, ax = plt.subplots(figsize=(16, 9))

colors = plt.cm.tab10(np.linspace(0, 1, len(MODELOS)))

for i, modelo in enumerate(MODELOS):
    # Calcular promedio global por N_Total (ignorando tipo de proceso)
    promedios_globales = df.groupby('N_Total')[modelo].mean()
    
    ax.plot(promedios_globales.index, promedios_globales.values, 
            marker='o', markersize=6, linewidth=2.5, 
            color=colors[i], label=modelo)

ax.set_xlabel('N_Total (Tamaño de Muestra)', fontsize=14)
ax.set_ylabel('ECRPS Promedio Global', fontsize=14)
ax.set_title('Tendencia Global: ECRPS Promedio por Modelo vs N_Total',
             fontsize=18, fontweight='bold', pad=20)

# Ajustar leyenda y grid
ax.legend(title='Modelos', title_fontsize=12, fontsize=11, 
          loc='upper left', bbox_to_anchor=(1, 1))
ax.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

archivo_global = output_dir / "tendencia_global_modelos.png"
plt.savefig(archivo_global, dpi=300, bbox_inches='tight')
print(f"Gráfica de tendencia global guardada: {archivo_global}")
plt.close()

# ============================================================================
# ANÁLISIS 6: RANKINGS Y FRECUENCIAS
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 6: Generando Rankings")
print("="*80)

rankings = []
for tipo_proceso in df['Tipo_Proceso'].unique():
    for ntotal in sorted(df['N_Total'].unique()):
        df_filtrado = df[(df['Tipo_Proceso'] == tipo_proceso) & (df['N_Total'] == ntotal)]
        
        errores_promedio = {}
        for modelo in MODELOS:
            errores_promedio[modelo] = df_filtrado[modelo].mean()
        
        ranking_modelos = sorted(errores_promedio.items(), key=lambda x: x[1])
        
        for rank, (modelo, score) in enumerate(ranking_modelos, 1):
            rankings.append({
                'Tipo_Proceso': tipo_proceso,
                'N_Total': ntotal,
                'Rank': rank,
                'Modelo': modelo,
                'ECRPS_Promedio': score
            })

df_rankings = pd.DataFrame(rankings)
archivo_rankings = output_dir / "rankings_modelos_por_ntotal.xlsx"
df_rankings.to_excel(archivo_rankings, index=False)

# ============================================================================
# ANÁLISIS 7: VARIABILIDAD (Heatmaps STD)
# ============================================================================

print("\n" + "="*80)
print("ANÁLISIS 7: Variabilidad (Desviación Estándar)")
print("="*80)

variabilidad_resultados = []

# Calcular variabilidad general
for modelo in MODELOS:
    for ntotal in sorted(df['N_Total'].unique()):
        vals = df[df['N_Total'] == ntotal][modelo]
        variabilidad_resultados.append({
            'Escenario': 'General',
            'Modelo': modelo,
            'N_Total': ntotal,
            'Desv_Std': vals.std(),
            'CV': (vals.std() / vals.mean() * 100) if vals.mean() != 0 else 0
        })

df_variabilidad = pd.DataFrame(variabilidad_resultados)

# Generar Heatmap de Desviación Estándar (General)
df_esc = df_variabilidad[df_variabilidad['Escenario'] == 'General']
pivot_std = df_esc.pivot(index='Modelo', columns='N_Total', values='Desv_Std')

fig, ax = plt.subplots(figsize=(14, 10))

sns.heatmap(pivot_std, 
            annot=True, 
            fmt='.2f',  # <--- CAMBIO: Solo 2 dígitos
            cmap='YlOrRd',
            cbar_kws={'label': 'Desviación Estándar (ECRPS)', 'shrink': 0.6}, # <--- CAMBIO: Barra reducida
            linewidths=0.5,
            linecolor='gray',
            annot_kws={'size': 9},
            square=True,
            ax=ax)

plt.title('Variabilidad (Desviación Estándar) del ECRPS\nEscenario General',
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()

archivo_var = output_dir / "variabilidad_std_general.png"
plt.savefig(archivo_var, dpi=300, bbox_inches='tight')
print(f"Heatmap de variabilidad (STD) guardado: {archivo_var}")
plt.close()

# ============================================================================
# FINALIZAR
# ============================================================================
print("\n✅ Proceso completado. Revisa la carpeta de resultados.")
gc.collect()

Cargando datos...
Columnas disponibles: ['Paso', 'Proceso', 'Tipo_Proceso', 'Distribución', 'Varianza', 'N_Train', 'N_Calib', 'N_Total', 'Valor_Observado', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'ESCENARIO']
N_Total únicos: [np.int64(120), np.int64(140), np.int64(160), np.int64(200), np.int64(220), np.int64(240), np.int64(260), np.int64(300), np.int64(320), np.int64(340), np.int64(360), np.int64(400), np.int64(500), np.int64(520), np.int64(540), np.int64(560), np.int64(600), np.int64(700), np.int64(1020), np.int64(1040), np.int64(1060), np.int64(1100), np.int64(1200)]
Número de observaciones: 126000

ANÁLISIS 1: HEATMAPS - Promedio de ECRPS por Proceso y N_Total
Heatmap guardado: resultados_analisis_ntotal\heatmap_AREPD.png
Heatmap guardado: resultados_analisis_ntotal\heatmap_AV-MCPS.png
Heatmap guardado: resultados_analisis_ntotal\heatmap_Block_Bootstrapping.png
Heatmap guardado: resultados_analisis_ntotal\heatmap

8451